In [2]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.sql.expression import cast
from sqlalchemy import create_engine, inspect, func
from sqlalchemy.orm import Session, aliased
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import datetime


In [3]:
# Create an engine for the `airbnb.db` database
### BEGIN SOLUTION
engine = create_engine("sqlite:///DC_airbnb.db", echo=False)
### END SOLUTION

In [4]:
# Use the Inspector to explore the database and print the table names
### BEGIN SOLUTION
inspector = inspect(engine)
inspector.get_table_names()
### END SOLUTION

['crime_table', 'listings_table']

In [5]:
# Use Inspector to print the column names and types
### BEGIN SOLUTION
columns = inspector.get_columns('listings_table')
for c in columns:
    print(c['name'], c["type"])
### END SOLUTION

id INTEGER
airbnb_id INTEGER
name VARCHAR(255)
host_id INTEGER
host_name VARCHAR(255)
neighbourhood VARCHAR(255)
latitude FLOAT
longitude FLOAT
room_type VARCHAR(255)
price INTEGER
minimum_nights INTEGER
number_of_reviews INTEGER
reviews_per_month FLOAT
calculated_host_listings_count INTEGER
availability_365 INTEGER


In [6]:
engine.execute('SELECT * FROM listings_table LIMIT 5').fetchall()

[(1, 4873463, b'\x00\x00\x00\x00\x00\x00\x00\x00', 7220554, 'TheoDora', 'Dupont Circle, Connecticut Avenue,K Street', 38.91518263, -77.03925945, 'Entire home,apt', 95, 2, 29, 1.01, 3, 53),
 (2, 16736650, b'\x01\x00\x00\x00\x00\x00\x00\x00', 47945335, 'Nina', 'Dupont Circle, Connecticut Avenue,K Street', 38.91258032, -77.04272065, 'Entire home,apt', 200, 1, 2, 0.55, 1, 0),
 (3, 14999877, b'\x02\x00\x00\x00\x00\x00\x00\x00', 10325135, 'Rory', 'Dupont Circle, Connecticut Avenue,K Street', 38.91706161, -77.03586862, 'Entire home,apt', 100, 27, 0, 0.0, 1, 0),
 (4, 5955860, b'\x03\x00\x00\x00\x00\x00\x00\x00', 3873893, 'Feras', 'Dupont Circle, Connecticut Avenue,K Street', 38.90981485, -77.04612379, 'Entire home,apt', 129, 2, 79, 3.13, 2, 324),
 (5, 15655208, b'\x04\x00\x00\x00\x00\x00\x00\x00', 73613929, 'Courtney', 'Dupont Circle, Connecticut Avenue,K Street', 38.91675913, -77.04034922, 'Entire home,apt', 500, 2, 1, 1.0, 1, 81)]

In [7]:

columns = inspector.get_columns('crime_table')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
neighirhood_cluster VARCHAR(225)
number_of_crimes INTEGER
neighbourhood VARCHAR(255)
crime_rate FLOAT


In [8]:
engine.execute('SELECT * FROM crime_table ').fetchall()

[(1, 'Cluster 18', 1411, 'Brightwood Park, Crestwood, Petworth', 4.314456947162427),
 (2, 'Cluster 22', 1244, 'Brookland, Brentwood, Langdon', 3.803816046966732),
 (3, 'Cluster 26', 1461, 'Capitol Hill, Lincoln Park', 4.467343444227006),
 (4, 'Cluster 33', 923, 'Capitol View, Marshall Heights, Benning Heights', 2.8222847358121337),
 (5, 'Cluster 14', 235, 'Cathedral Heights, McLean Gardens, Glover Park', 0.7185665362035225),
 (6, 'Cluster 15', 328, 'Cleveland Park, Woodley Park, Massachusetts Avenue Heights, Woodland-Normanstone Terrace', 1.0029354207436398),
 (7, 'Cluster 16', 150, 'Colonial Village, Shepherd Park, North Portal Estates', 0.4586594911937377),
 (8, 'Cluster 2', 2654, 'Columbia Heights, Mt. Pleasant, Pleasant Plains, Park View', 8.115215264187867),
 (9, 'Cluster 39', 1284, 'Congress Heights, Bellevue, Washington Highlands', 3.926125244618396),
 (10, 'Cluster 31', 950, 'Deanwood, Burrville, Grant Park, Lincoln Heights, Fairmont Heights', 2.9048434442270064),
 (11, 'Cluste

In [9]:
# Declare a Base using `automap_base()`
Base = automap_base()

In [10]:
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [11]:
# Print all of the classes mapped to the Base
Base.classes.keys()

['crime_table', 'listings_table']

In [12]:
Listings = Base.classes.listings_table

In [13]:
Crime=Base.classes.crime_table

In [14]:
# Create a session
session = Session(engine)

In [15]:

neighbourhood_results = session.query(Listings.neighbourhood).group_by(Listings.neighbourhood).all()
neighbourhood_results

[('Brightwood Park, Crestwood, Petworth'),
 ('Brookland, Brentwood, Langdon'),
 ('Capitol Hill, Lincoln Park'),
 ('Capitol View, Marshall Heights, Benning Heights'),
 ('Cathedral Heights, McLean Gardens, Glover Park'),
 ('Cleveland Park, Woodley Park, Massachusetts Avenue Heights, Woodland-Normanstone Terrace'),
 ('Colonial Village, Shepherd Park, North Portal Estates'),
 ('Columbia Heights, Mt. Pleasant, Pleasant Plains, Park View'),
 ('Congress Heights, Bellevue, Washington Highlands'),
 ('Deanwood, Burrville, Grant Park, Lincoln Heights, Fairmont Heights'),
 ('Douglas, Shipley Terrace'),
 ('Downtown, Chinatown, Penn Quarters, Mount Vernon Square, North Capitol Street'),
 ('Dupont Circle, Connecticut Avenue,K Street'),
 ('Eastland Gardens, Kenilworth'),
 ('Edgewood, Bloomingdale, Truxton Circle, Eckington'),
 ('Fairfax Village, Naylor Gardens, Hillcrest, Summit Park'),
 ('Friendship Heights, American University Park, Tenleytown'),
 ('Georgetown, Burleith,Hillandale'),
 ('Hawthorne, B

In [16]:
neighbourhood= "Dupont Circle, Connecticut Avenue,K Street"
Entire_home_Listings_results = session.query(Listings.neighbourhood,func.avg(Listings.price ),\
                func.avg(Listings.number_of_reviews ),  func.avg(Listings.reviews_per_month ),\
                func.avg(Listings.availability_365)).filter(Listings.room_type == 'Entire home,apt').\
                filter(Listings.neighbourhood == neighbourhood).group_by(Listings.neighbourhood).all()
Entire_home_Listings_results

[('Dupont Circle, Connecticut Avenue,K Street',
  276.57710280373834,
  25.10514018691589,
  1.4347196261682227,
  124.30373831775701)]

In [17]:

Entire_home_price_list=[]
for list in Entire_home_Listings_results:
    row = {"neighbourhood":"price", "neighbourhood": "review_numbers","neighbourhood": "reviews_per_month", "neighbourhood":"availability_365" }
    row["neighbourhood"] = list[0]
    row["price"] = list[1]
    row["review_numbers"] = list[2]
    row["reviews_per_month"] = list[3]
    row["availability_365"] = list[4]
    Entire_home_price_list.append(row)
Entire_home_price_list    

[{'availability_365': 124.30373831775701,
  'neighbourhood': 'Dupont Circle, Connecticut Avenue,K Street',
  'price': 276.57710280373834,
  'review_numbers': 25.10514018691589,
  'reviews_per_month': 1.4347196261682227}]

In [18]:

Private_room_Listings_results = session.query(Listings.neighbourhood,func.avg(Listings.price ),\
                func.avg(Listings.number_of_reviews ),func.avg(Listings.reviews_per_month ), func.avg(Listings.availability_365)).\
                filter(Listings.room_type == 'Private room').group_by(Listings.neighbourhood).all()
Private_room_Listings_results

[('Brightwood Park, Crestwood, Petworth',
  88.4931506849315,
  16.815068493150687,
  1.2645890410958904,
  149.93150684931507),
 ('Brookland, Brentwood, Langdon', 90.52, 13.0, 1.1450000000000002, 146.5),
 ('Capitol Hill, Lincoln Park',
  141.8130081300813,
  34.45528455284553,
  2.030813008130081,
  127.92682926829268),
 ('Capitol View, Marshall Heights, Benning Heights',
  114.13513513513513,
  13.027027027027026,
  1.1351351351351349,
  224.9189189189189),
 ('Cathedral Heights, McLean Gardens, Glover Park',
  118.72972972972973,
  19.324324324324323,
  0.9305405405405406,
  125.67567567567568),
 ('Cleveland Park, Woodley Park, Massachusetts Avenue Heights, Woodland-Normanstone Terrace',
  133.0,
  16.3125,
  1.5075,
  118.0),
 ('Colonial Village, Shepherd Park, North Portal Estates',
  108.93333333333334,
  9.4,
  0.5399999999999998,
  274.2),
 ('Columbia Heights, Mt. Pleasant, Pleasant Plains, Park View',
  91.65612648221344,
  20.26086956521739,
  1.3118181818181816,
  127.4664031

In [19]:
Private_room_price_list=[]
for list in Private_room_Listings_results:
    row = {"neighbourhood":"price", "neighbourhood": "review_numbers","neighbourhood": "reviews_per_month", "neighbourhood":"availability_365" }
    row["neighbourhood"] = list[0]
    row["price"] = list[1]
    row["review_numbers"] = list[2]
    row["reviews_per_month"] = list[3]
    row["availability_365"] = list[4]
    Private_room_price_list.append(row)
Private_room_price_list   

[{'availability_365': 149.93150684931507,
  'neighbourhood': 'Brightwood Park, Crestwood, Petworth',
  'price': 88.4931506849315,
  'review_numbers': 16.815068493150687,
  'reviews_per_month': 1.2645890410958904},
 {'availability_365': 146.5,
  'neighbourhood': 'Brookland, Brentwood, Langdon',
  'price': 90.52,
  'review_numbers': 13.0,
  'reviews_per_month': 1.1450000000000002},
 {'availability_365': 127.92682926829268,
  'neighbourhood': 'Capitol Hill, Lincoln Park',
  'price': 141.8130081300813,
  'review_numbers': 34.45528455284553,
  'reviews_per_month': 2.030813008130081},
 {'availability_365': 224.9189189189189,
  'neighbourhood': 'Capitol View, Marshall Heights, Benning Heights',
  'price': 114.13513513513513,
  'review_numbers': 13.027027027027026,
  'reviews_per_month': 1.1351351351351349},
 {'availability_365': 125.67567567567568,
  'neighbourhood': 'Cathedral Heights, McLean Gardens, Glover Park',
  'price': 118.72972972972973,
  'review_numbers': 19.324324324324323,
  'rev

In [20]:
Shared_room_Listings_results = session.query(Listings.neighbourhood,func.avg(Listings.price ),\
                func.avg(Listings.number_of_reviews ), func.avg(Listings.reviews_per_month ),func.avg(Listings.availability_365)).\
                filter(Listings.room_type == 'Shared room').group_by(Listings.neighbourhood).all()
Shared_room_Listings_results

[('Brightwood Park, Crestwood, Petworth', 72.0, 3.8, 0.256, 266.6),
 ('Brookland, Brentwood, Langdon', 93.5, 11.0, 0.6, 364.5),
 ('Capitol Hill, Lincoln Park',
  213.71428571428572,
  6.285714285714286,
  0.8371428571428571,
  100.71428571428571),
 ('Capitol View, Marshall Heights, Benning Heights',
  15.0,
  5.5,
  0.9283333333333333,
  358.8333333333333),
 ('Cathedral Heights, McLean Gardens, Glover Park',
  35.875,
  0.625,
  0.2025,
  32.5),
 ('Cleveland Park, Woodley Park, Massachusetts Avenue Heights, Woodland-Normanstone Terrace',
  55.0,
  14.5,
  1.695,
  57.5),
 ('Columbia Heights, Mt. Pleasant, Pleasant Plains, Park View',
  43.18181818181818,
  14.272727272727273,
  1.8513636363636365,
  152.0909090909091),
 ('Congress Heights, Bellevue, Washington Highlands', 60.0, 0.0, 0.0, 78.5),
 ('Deanwood, Burrville, Grant Park, Lincoln Heights, Fairmont Heights',
  38.666666666666664,
  0.0,
  0.0,
  72.66666666666667),
 ('Downtown, Chinatown, Penn Quarters, Mount Vernon Square, Nort

In [21]:
Shared_room_price_list=[]
for list in Shared_room_Listings_results:
    row = {"neighbourhood":"price", "neighbourhood": "review_numbers", "neighbourhood":"availability_365" }
    row["neighbourhood"] = list[0]
    row["price"] = list[1]
    row["review_numbers"] = list[2]
    row["reviews_per_month"] = list[3]
    row["availability_365"] = list[4]
    Shared_room_price_list.append(row)
Shared_room_price_list    

[{'availability_365': 266.6,
  'neighbourhood': 'Brightwood Park, Crestwood, Petworth',
  'price': 72.0,
  'review_numbers': 3.8,
  'reviews_per_month': 0.256},
 {'availability_365': 364.5,
  'neighbourhood': 'Brookland, Brentwood, Langdon',
  'price': 93.5,
  'review_numbers': 11.0,
  'reviews_per_month': 0.6},
 {'availability_365': 100.71428571428571,
  'neighbourhood': 'Capitol Hill, Lincoln Park',
  'price': 213.71428571428572,
  'review_numbers': 6.285714285714286,
  'reviews_per_month': 0.8371428571428571},
 {'availability_365': 358.8333333333333,
  'neighbourhood': 'Capitol View, Marshall Heights, Benning Heights',
  'price': 15.0,
  'review_numbers': 5.5,
  'reviews_per_month': 0.9283333333333333},
 {'availability_365': 32.5,
  'neighbourhood': 'Cathedral Heights, McLean Gardens, Glover Park',
  'price': 35.875,
  'review_numbers': 0.625,
  'reviews_per_month': 0.2025},
 {'availability_365': 57.5,
  'neighbourhood': 'Cleveland Park, Woodley Park, Massachusetts Avenue Heights, W

In [22]:
crime_results = session.query(Crime.neighbourhood, Crime.number_of_crimes, Crime.crime_rate).\
                group_by(Crime.neighbourhood).order_by(Crime.number_of_crimes.desc()).all()
crime_results        

[('Columbia Heights, Mt. Pleasant, Pleasant Plains, Park View',
  2654,
  8.115215264187867),
 ('Downtown, Chinatown, Penn Quarters, Mount Vernon Square, North Capitol Street',
  2307,
  7.054182974559686),
 ('Union Station, Stanton Park, Kingman Park', 1918, 5.8647260273972615),
 ('Dupont Circle, Connecticut Avenue,K Street', 1605, 4.907656555772994),
 ('Capitol Hill, Lincoln Park', 1461, 4.467343444227006),
 ('Howard University, Le Droit Park, Cardozo,Shaw', 1459, 4.461227984344423),
 ('Brightwood Park, Crestwood, Petworth', 1411, 4.314456947162427),
 ('Ivy City, Arboretum, Trinidad, Carver Langston', 1403, 4.2899951076320955),
 ('Congress Heights, Bellevue, Washington Highlands', 1284, 3.926125244618396),
 ('Brookland, Brentwood, Langdon', 1244, 3.803816046966732),
 ('Shaw, Logan Circle', 1169, 3.5744863013698627),
 ('Edgewood, Bloomingdale, Truxton Circle, Eckington',
  1103,
  3.3726761252446185),
 ('Georgetown, Burleith,Hillandale', 1064, 3.2534246575342465),
 ('Deanwood, Burrvil

In [23]:

crime_list=[]
for crime in crime_results:
    row = {"neighbourhood":"crime_counts", "neighbourhood":"crime_rate"}
    row["neighbourhood"] = crime[0]
    row["crime_counts"] = crime[1]
    row["crime_rate"] = crime[2]
    crime_list.append(row)
crime_list      

[{'crime_counts': 2654,
  'crime_rate': 8.115215264187867,
  'neighbourhood': 'Columbia Heights, Mt. Pleasant, Pleasant Plains, Park View'},
 {'crime_counts': 2307,
  'crime_rate': 7.054182974559686,
  'neighbourhood': 'Downtown, Chinatown, Penn Quarters, Mount Vernon Square, North Capitol Street'},
 {'crime_counts': 1918,
  'crime_rate': 5.8647260273972615,
  'neighbourhood': 'Union Station, Stanton Park, Kingman Park'},
 {'crime_counts': 1605,
  'crime_rate': 4.907656555772994,
  'neighbourhood': 'Dupont Circle, Connecticut Avenue,K Street'},
 {'crime_counts': 1461,
  'crime_rate': 4.467343444227006,
  'neighbourhood': 'Capitol Hill, Lincoln Park'},
 {'crime_counts': 1459,
  'crime_rate': 4.461227984344423,
  'neighbourhood': 'Howard University, Le Droit Park, Cardozo,Shaw'},
 {'crime_counts': 1411,
  'crime_rate': 4.314456947162427,
  'neighbourhood': 'Brightwood Park, Crestwood, Petworth'},
 {'crime_counts': 1403,
  'crime_rate': 4.2899951076320955,
  'neighbourhood': 'Ivy City, Ar

In [24]:
All_Listings_results = session.query(Listings.neighbourhood, Listings.price, Listings.airbnb_id, Listings.name, Listings.host_id,Listings.host_name, Listings.room_type,Listings.minimum_nights,Listings.number_of_reviews,Listings.reviews_per_month, Listings.availability_365).all()

In [25]:
All_Listings_results

[('Dupont Circle, Connecticut Avenue,K Street',
  95,
  4873463,
  b'\x00\x00\x00\x00\x00\x00\x00\x00',
  7220554,
  'TheoDora',
  'Entire home,apt',
  2,
  29,
  1.01,
  53),
 ('Dupont Circle, Connecticut Avenue,K Street',
  200,
  16736650,
  b'\x01\x00\x00\x00\x00\x00\x00\x00',
  47945335,
  'Nina',
  'Entire home,apt',
  1,
  2,
  0.55,
  0),
 ('Dupont Circle, Connecticut Avenue,K Street',
  100,
  14999877,
  b'\x02\x00\x00\x00\x00\x00\x00\x00',
  10325135,
  'Rory',
  'Entire home,apt',
  27,
  0,
  0.0,
  0),
 ('Dupont Circle, Connecticut Avenue,K Street',
  129,
  5955860,
  b'\x03\x00\x00\x00\x00\x00\x00\x00',
  3873893,
  'Feras',
  'Entire home,apt',
  2,
  79,
  3.13,
  324),
 ('Dupont Circle, Connecticut Avenue,K Street',
  500,
  15655208,
  b'\x04\x00\x00\x00\x00\x00\x00\x00',
  73613929,
  'Courtney',
  'Entire home,apt',
  2,
  1,
  1.0,
  81),
 ('Dupont Circle, Connecticut Avenue,K Street',
  110,
  4022565,
  b'\x05\x00\x00\x00\x00\x00\x00\x00',
  5487930,
  'Mark',


In [26]:
df_data = pd.read_csv("clean_listing_data.csv")
grouped_cleanlist_NH= df_data.groupby(['neighbourhood'])
print(grouped_cleanlist_NH)
price_data=[]
price_data_dict={}
neighbourhood_name = "Dupont Circle, Connecticut Avenue,K Street"
for group in grouped_cleanlist_NH:
    
    AList = list(group[1].neighbourhood)
    BList = list(group[1].airbnb_id)
    CList = list(group[1].price)
    if AList[0]==neighbourhood_name:
        price_data_dict={"airbnb_ids": BList,"price":  CList}
        price_data.append(price_data_dict)
price_data[0]

TypeError: 'result' object is not callable

In [27]:
neighbourhood= "Dupont Circle, Connecticut Avenue,K Street"
crimedata = []
crime_results = session.query(Crime.neighbourhood, \
            Crime.number_of_crimes, Crime.crime_rate).\
            group_by(Crime.neighbourhood).filter(Crime.neighbourhood == neighbourhood).all()
for result in crime_results :
        crimedata_dict = {}
        crimedata_dict["Neighbourhood"] = result[0]
        crimedata_dict["Number_of_Crimes"] = result[1]
        crimedata_dict["Crime_Rate"] = result[2]
    
        crimedata.append(crimedata_dict)
crimedata         

[{'Crime_Rate': 4.907656555772994,
  'Neighbourhood': 'Dupont Circle, Connecticut Avenue,K Street',
  'Number_of_Crimes': 1605}]

In [28]:
neighbourhood= "Dupont Circle, Connecticut Avenue,K Street"
Price_Listings_results = session.query(Listings.price, Listings.airbnb_id, Listings.number_of_reviews).\
                filter(Listings.neighbourhood == neighbourhood).group_by(Listings.neighbourhood).all()
Price_Listings_results 

[(70, 15883590, 3)]

In [29]:
Price_Listings= []
for result in Price_Listings_results :
        row = {"airbnb_id":"price", "airbnb_id":"number_of_reviews"}
        row["airbnb_id"] = result[1]
        row["price"] = result[0]
        row["number_of_reviews"] = result[2]
        Price_Listings.append(row)
Price_Listings

[{'airbnb_id': 15883590, 'number_of_reviews': 3, 'price': 70}]

In [30]:
neighbourhood= "Dupont Circle, Connecticut Avenue,K Street"


price_results = session.query(Listings.price).group_by(Listings.neighbourhood).all()
price_results 
   

[(120),
 (60),
 (95),
 (0),
 (249),
 (99),
 (115),
 (79),
 (75),
 (33),
 (75),
 (500),
 (70),
 (205),
 (70),
 (37),
 (155),
 (198),
 (89),
 (54),
 (30),
 (95),
 (65),
 (85),
 (50),
 (500),
 (170),
 (240),
 (150),
 (39),
 (95),
 (850),
 (225),
 (175),
 (51),
 (115),
 (25),
 (423),
 (72)]

In [31]:
price =  np.ravel(price_results )
price

array([120,  60,  95,   0, 249,  99, 115,  79,  75,  33,  75, 500,  70,
       205,  70,  37, 155, 198,  89,  54,  30,  95,  65,  85,  50, 500,
       170, 240, 150,  39,  95, 850, 225, 175,  51, 115,  25, 423,  72])

In [32]:
stmt = session.query(Listings).statement
df = pd.read_sql_query(stmt, session.bind)

    # Make sure that the sample was found in the columns, else throw an error
df

,id,airbnb_id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,1,4873463,b'\x00\x00\x00\x00\x00\x00\x00\x00',7220554,TheoDora,"Dupont Circle, Connecticut Avenue,K Street",38.915183,-77.039259,"Entire home,apt",95,2,29,1.01,3,53
1,2,16736650,b'\x01\x00\x00\x00\x00\x00\x00\x00',47945335,Nina,"Dupont Circle, Connecticut Avenue,K Street",38.912580,-77.042721,"Entire home,apt",200,1,2,0.55,1,0
2,3,14999877,b'\x02\x00\x00\x00\x00\x00\x00\x00',10325135,Rory,"Dupont Circle, Connecticut Avenue,K Street",38.917062,-77.035869,"Entire home,apt",100,27,0,0.00,1,0
3,4,5955860,b'\x03\x00\x00\x00\x00\x00\x00\x00',3873893,Feras,"Dupont Circle, Connecticut Avenue,K Street",38.909815,-77.046124,"Entire home,apt",129,2,79,3.13,2,324
4,5,15655208,b'\x04\x00\x00\x00\x00\x00\x00\x00',73613929,Courtney,"Dupont Circle, Connecticut Avenue,K Street",38.916759,-77.040349,"Entire home,apt",500,2,1,1.00,1,81
5,6,4022565,b'\x05\x00\x00\x00\x00\x00\x00\x00',5487930,Mark,"Dupont Circle, Connecticut Avenue,K Street",38.915450,-77.040203,Private room,110,6,7,0.23,13,357
6,7,15968425,b'\x06\x00\x00\x00\x00\x00\x00\x00',23211126,Henry,"Dupont Circle, Connecticut Avenue,K Street",38.915185,-77.037679,"Entire home,apt",225,3,3,0.82,1,48
7,8,1301700,b'\x07\x00\x00\x00\x00\x00\x00\x00',7077536,Brett,"Dupont Circle, Connecticut Avenue,K Street",38.906845,-77.034755,"Entire home,apt",79,3,4,0.09,1,0
8,9,646183,b'\x08\x00\x00\x00\x00\x00\x00\x00',3104115,Olivier,"Dupont Circle, Connecticut Avenue,K Street",38.917344,-77.038756,"Entire home,apt",172,1,24,0.48,1,0
9,10,14769106,b'\t\x00\x00\x00\x00\x00\x00\x00',92131371,Evan And Sabrina,"Dupont Circle, Connecticut Avenue,K Street",38.912423,-77.038127,Private room,110,1,4,0.67,1,10


In [33]:
neighbourhood= "Dupont Circle, Connecticut Avenue,K Street"

for i in df["neighbourhood"]:
    if i==neighbourhood:
        print(i)

Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circle, Connecticut Avenue,K Street
Dupont Circ

In [34]:
neighbourhood= "Dupont Circle, Connecticut Avenue,K Street"
if neighbourhood not in df["neighbourhood"]:
    #return (f"Error! Listings: {neighbourhood} Not Found!")

    df = df.sort_values(by="price", ascending=0)
 # Format the data to send as json
    data = [{
        "airbnb_ids": df["airbnb_id"].values.tolist(),
        "price": df["price"].values.tolist(),
        "number_of_reviews": df["number_of_reviews"].values.tolist(),
        "host_id": df["host_id"].values.tolist(),
        "calculated_host_listings_count": df["calculated_host_listings_count"].values.tolist(),
        "room_type": df["room_type"].values.tolist(),
        "minimum_nights": df["minimum_nights"].values.tolist(),
        "availability_365": df["availability_365"].values.tolist(),

    }]
print(data )

[{'airbnb_ids': [16243967, 14507861, 16671797, 16634189, 15151751, 16661573, 16769736, 16440243, 16532102, 16741673, 16436737, 16376320, 16122861, 16185167, 16466316, 16739434, 16146065, 16352224, 16709404, 12017374, 16228569, 16477504, 16215109, 16514478, 16042165, 16158966, 16225583, 15806228, 16792949, 16254327, 15111597, 16649224, 14022330, 16488083, 16322835, 16757379, 16322236, 16480798, 15964877, 16237886, 16274018, 16190668, 16284165, 16047690, 16052032, 16164164, 16096687, 16439413, 15755806, 16123903, 2070286, 16352908, 16441766, 16242411, 16053784, 16406885, 16061725, 16160575, 16364273, 15941087, 16026199, 16488882, 16427050, 16534993, 16314039, 16068448, 8311492, 16780821, 16585195, 16756920, 16620516, 15940945, 16475856, 16192278, 16309703, 16427233, 16598388, 16740002, 16219662, 16049905, 16442358, 16332673, 16148756, 16189905, 8285534, 16335002, 865732, 16094050, 16505075, 16155048, 16260977, 16212491, 16805493, 16408669, 15295591, 16086334, 12682227, 16050016, 16803176

In [36]:
neighbourhood= "Dupont Circle, Connecticut Avenue,K Street"
eachNH_df=df.loc[df['neighbourhood'] == neighbourhood]

eachNH_df   

,id,airbnb_id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
3099,3100,16739434,b'\x1b\x0c\x00\x00\x00\x00\x00\x00',24876170,Bryan,"Dupont Circle, Connecticut Avenue,K Street",38.909329,-77.045549,"Entire home,apt",3500,3,0,0.00,1,0
3149,3150,16228569,b'M\x0c\x00\x00\x00\x00\x00\x00',50775873,Janet,"Dupont Circle, Connecticut Avenue,K Street",38.913300,-77.037798,"Entire home,apt",3200,3,0,0.00,1,0
3310,3311,16740002,b'\xee\x0c\x00\x00\x00\x00\x00\x00',92363337,Sean,"Dupont Circle, Connecticut Avenue,K Street",38.908299,-77.038408,"Entire home,apt",2000,1,0,0.00,1,0
82,83,16335002,b'R\x00\x00\x00\x00\x00\x00\x00',1638940,Faisal,"Dupont Circle, Connecticut Avenue,K Street",38.911755,-77.035453,"Entire home,apt",2000,3,0,0.00,1,179
3324,3325,16803176,b'\xfc\x0c\x00\x00\x00\x00\x00\x00',89866028,Marissa,"Dupont Circle, Connecticut Avenue,K Street",38.909129,-77.036014,"Entire home,apt",1925,1,0,0.00,1,0
10,11,16669509,b'\n\x00\x00\x00\x00\x00\x00\x00',110267232,John,"Dupont Circle, Connecticut Avenue,K Street",38.912123,-77.035420,"Entire home,apt",1700,2,0,0.00,1,90
3031,3032,16635016,b'\xd7\x0b\x00\x00\x00\x00\x00\x00',14137964,Spencer,"Dupont Circle, Connecticut Avenue,K Street",38.910395,-77.035210,"Entire home,apt",1500,1,0,0.00,1,0
3268,3269,15858858,b'\xc4\x0c\x00\x00\x00\x00\x00\x00',72555356,Art,"Dupont Circle, Connecticut Avenue,K Street",38.913562,-77.038731,"Entire home,apt",1400,2,0,0.00,2,98
3322,3323,16084499,b'\xfa\x0c\x00\x00\x00\x00\x00\x00',44882389,Hunter,"Dupont Circle, Connecticut Avenue,K Street",38.915304,-77.040252,"Entire home,apt",1300,2,0,0.00,1,0
3101,3102,16225988,b'\x1d\x0c\x00\x00\x00\x00\x00\x00',23350298,Natalie,"Dupont Circle, Connecticut Avenue,K Street",38.912259,-77.044357,"Entire home,apt",1200,2,0,0.00,1,0


In [38]:
eachNH_df= eachNH_df.sort_values(by="price", ascending=0)
data = [{
        "airbnb_ids": eachNH_df ["airbnb_id"].values.tolist(),
        "price": eachNH_df ["price"].values.tolist(),
        "number_of_reviews": eachNH_df ["number_of_reviews"].values.tolist(),
        "host_id": eachNH_df ["host_id"].values.tolist(),
        "calculated_host_listings_count":eachNH_df ["calculated_host_listings_count"].values.tolist(),
        "room_type": eachNH_df ["room_type"].values.tolist(),
        "minimum_nights": eachNH_df ["minimum_nights"].values.tolist(),
        "availability_365": eachNH_df ["availability_365"].values.tolist(),

    }]
print(data )

[{'airbnb_ids': [16739434, 16228569, 16740002, 16335002, 16803176, 16669509, 16635016, 15858858, 16084499, 16225988, 16164802, 16615538, 16236713, 15870714, 16143403, 16240547, 16477855, 16806661, 789295, 16015764, 16217721, 13513389, 16748305, 16859999, 14236814, 16602888, 16553448, 16674846, 16041410, 10568730, 16725978, 16350934, 16718716, 16869795, 16311457, 16456318, 16618281, 16122969, 16177069, 15820404, 16465756, 16269755, 16422635, 8291283, 16614815, 16243506, 16705033, 8503628, 851048, 16674057, 13215892, 16732951, 16268194, 16427840, 16688556, 6807421, 16159729, 16350541, 15998469, 15834499, 16481119, 15655208, 7509292, 15925363, 16770871, 16583499, 14346963, 11851356, 16532376, 16571149, 16748395, 8530589, 16969476, 16664332, 12833506, 16644919, 16467167, 3897881, 16755909, 8390124, 16122617, 16645279, 11825363, 6011730, 14788134, 16250579, 15327360, 16324805, 16395596, 16242812, 16630454, 16806095, 16449357, 16741892, 16377238, 13413787, 16844228, 684324, 2429982, 16794336